In [9]:
#Import relevant dataset
import pickle as pk
import pandas as pd
import networkx as nx
import numpy as np

In [10]:
#read the dataset
authordf = pd.read_pickle("disambiguated-author-dataframe-1991-2010-v2-modified.pkl")

In [11]:
#check the head of the dataset
authordf.head()

,paperlist,paperdict,numpapers,yearsactive,numyearsactive,firstyear,lastyear,careerlength,numpapersperyear,coauthors,...,affiliationlist,numaffiliations,codes,numuniquecodes,numcodesperpaper,journals,numuniquejournals,affiliationbyyear,disambafflist,afflistnoand
a.;abbott,"[0588692, 0513160, 0400048, 0422137, 0618667]","{u'0422137': [u'1997', u'English', [u'Abbott, ...",5,"[1999, 2002, 2000, 1997, 1996]",5,1996,2002,7,0.714286,"[Seddighi, H. R., De Vita, G., Armistead, Chri...",...,"[U Sunderland, St Peter's Campus, Sunderland B...",4,[Metals and Metal Products; Cement; Glass; Cer...,6,1.200000,"[Applied Economics, Economic Issues]",2,"{u'1999': [u'U Sunderland'], u'2002': [u'Oxfor...","[u sunderland, st peters campus, sunderland bu...","[u sunderland, st peters campus, sunderland bu..."
a.;abdul kalam,"[0743293, 0866863, 0728163, 0833297]","{u'0728163': [u'2003', u'English', [u'Abdul Ka...",4,"[2003, 2006, 2004, 2005]",4,2003,2006,4,1.000000,"[Abdul Kalam, A. P. J.]",...,"[Government of India, Government, India, Offic...",4,"[Models of Political Processes: Rent-seeking, ...",9,2.250000,[Finance India],1,"{u'2003': [u'Office of President of India'], u...","[government of india, government, india, offic...","[government of india, government, india, offic..."
a.;adair,"[1126382, 0535621]","{u'1126382': [u'2010', u'English', [u'Adair, A...",2,"[2000, 2010]",2,2000,2010,11,0.181818,"[Adair, A., Lloyd, M. G., Berry, J., McGreal, ...",...,"[Built Environment Research Institute, U Ulste...",2,"[Housing Supply and Markets_(R310), Business F...",6,3.000000,"[Urban Studies, Local Economy]",2,{u'2000': [u'Centre for Research on Property &...,"[built environment research institute, u ulste...","[built environment research institute, u ulste..."
a.;adams,"[0497129, 0638811, 1011448, 0604092, 0669492, ...","{u'1011448': [u'2008', u'English', [u'Adams, A...",6,"[1997, 1999, 2002, 2003, 2001, 2008]",6,1997,2008,12,0.500000,"[Kaserman, David L., Naydenova, Budina, Ekelun...",...,"[College of Charleston, Gustavus Adolphus Coll...",3,"[Tort Law and Product Liability_(K130), Econom...",13,2.166667,"[Journal of Labor Research, Journal of Forensi...",6,"{u'1997': [u'Gustavus Adolphus College'], u'19...","[college of charleston, gustavus adolphus coll...","[college of charleston, gustavus adolphus coll..."
a.;ades,"[0534842, 0952911, 1009098, 1009089, 1120368, ...","{u'1009089': [u'2008', u'English', [u'Welton, ...",11,"[2000, 2006, 2007, 2008, 2009, 2010]",6,2000,2010,11,1.000000,"[Carlin, J. B., Spiegelhalter, D. J., Jones, D...",...,"[Centre for Health Economics, U York, Medical ...",6,"[Health: General_(I100), Related Disciplines_(...",10,0.909091,[Journal of the Royal Statistical Society: Ser...,4,"{u'2000': [u'Institute of Child Health, London...","[centre for health economics, u york, medical ...","[centre for health economics, u york, medical ..."


In [12]:
for a in authordf.transpose():
    afflist = authordf['afflistnoand'][a]
    while '' in afflist:
        afflist.remove('')

In [13]:
authordf = authordf[authordf['afflistnoand'].map(len) > 1]

In [14]:
#define a function to check if two lists have common elements, return boolean.
def list_overlap(lst1, lst2):
    return any(i in lst1 for i in lst2)

In [15]:
keep = 0
drop = 0
keeplist = []
for a in authordf.transpose():
    afflist = authordf['afflistnoand'][a]
    first = afflist[0]
    final = afflist[len(afflist)-1]
    keeplisttemp = []
    #if first is string and final is lst
    if (type(first) == str and type(final) == list):
        if first in final:
            drop += 1
        else:
            keeplisttemp.append(first)
            keeplisttemp.append(final)
            keeplist.append(keeplisttemp)
            keep += 1
            
    #if first is list and final is string
    elif (type(first) == list and type(final) == str):
        if final in first:
            drop += 1
        else:
            keeplisttemp.append(first)
            keeplisttemp.append(final)
            keeplist.append(keeplisttemp)
            keep += 1                
                
    #if boths are string
    elif (type(first) == str and type(final) == str):
        if first == final:
            drop += 1
        else:
            keeplisttemp.append(first)
            keeplisttemp.append(final)
            keeplist.append(keeplisttemp)
            keep += 1
            
    #if boths are list
    elif (type(first) == list and type(final) == list):
        if list_overlap(first,final):
            drop += 1
        else:
            keeplisttemp.append(first)
            keeplisttemp.append(final)
            keeplist.append(keeplisttemp)
            keep += 1
    else:
        pass
print keep
print drop

69030
3337


In [16]:
affnwdict = nx.DiGraph()
for aff in keeplist:
    first = aff[0]
    final = aff[1]
    
    #if first is string and final is lst
    if (type(first) == str and type(final) == list):
        firstnode1 = first
        for subaff in final:
            if affnwdict.has_edge(firstnode1,subaff):
                old_wt = affnwdict[firstnode1][subaff]['weight']
                new_wt = old_wt + 1
                affnwdict.add_edge(firstnode1,subaff,weight = new_wt)
            else:
                affnwdict.add_edge(firstnode1,subaff,weight = 1)
                
    #if first is list and final is string
    elif (type(first) == list and type(final) == str):   
        finalnode1 = final
        for subaff in first:
            if affnwdict.has_edge(subaff,finalnode1):
                old_wt = affnwdict[subaff][finalnode1]['weight']
                new_wt = old_wt + 1
                affnwdict.add_edge(subaff,finalnode1,weight = new_wt)
            else:
                affnwdict.add_edge(subaff,finalnode1,weight = 1)
                
    #if boths are string
    elif (type(first) == str and type(final) == str):  
        firstnode = first
        finalnode = final
        if affnwdict.has_edge(firstnode,finalnode):
            old_wt = affnwdict[firstnode][finalnode]['weight']
            new_wt = old_wt + 1
            affnwdict.add_edge(firstnode,finalnode,weight = new_wt)
        else:
            affnwdict.add_edge(firstnode,finalnode,weight = 1)
            
    #if boths are list
    elif (type(first) == list and type(final) == list):
        for firstnode in first:
            for finalnode in final:
                if affnwdict.has_edge(firstnode,finalnode):
                    old_wt = affnwdict[firstnode][finalnode]['weight']
                    new_wt = old_wt + 1
                    affnwdict.add_edge(firstnode,finalnode,weight = new_wt)
                else:
                    affnwdict.add_edge(firstnode,finalnode,weight = 1)
    else:
        pass
        

In [17]:
affnwdict.nodes()

['europeancommission',
 'samsung economic research institute',
 'wirtschaftstheorie ii, u bonn',
 'u mi, flint',
 'centre for development studies, medical college trivandrum',
 'u antwerp-ufsia',
 'college of nj',
 'macgillivrays solicitors',
 'st anna school of advanced studies',
 'pdf llc',
 'australian bureau of agricultural and resource econ, canberra',
 'hebrew u and washington u',
 'helsinki, finland',
 'nv state office of energy',
 'imri, u paris dauphine',
 'ellicott city, md',
 'european parliament secretariat',
 'u piraeus',
 'aalborg u',
 'u quebec a trois-rivieres',
 'cathay united bank, taichung',
 'georg august u gottingen',
 'institute of international relations, u west indies',
 'national taiwan u science and technology and jin wen institute oftechnology, taiwan',
 'aston business school',
 'vienna u econoimcs and business administration',
 'southern ca association of governments, los angeles',
 'escuela juperior de agricultura de barcelona-upc',
 'u oxford, federal res

In [18]:
len(affnwdict.nodes())

35091

In [19]:
affnwdict.edges()

[('europeancommission', 'european commission'),
 ('u mi, flint',
  'u mi, flint and center for russian and east european studies, umi'),
 ('u mi, flint', 'skidmore college'),
 ('u mi, flint', 'mi state u'),
 ('u mi, flint', 'george mason u'),
 ('u mi, flint', 'lebanese american u'),
 ('u mi, flint', 'george washington u'),
 ('u mi, flint', 'grand valley state u'),
 ('u mi, flint', 'robert morris college'),
 ('u mi, flint', 'james madison college'),
 ('u mi, flint', 'sonoma state u'),
 ('u mi, flint', 'u mi flint'),
 ('u mi, flint', 'temple u'),
 ('u mi, flint', 'u toledo'),
 ('u mi, flint', 'austin peay u'),
 ('u mi, flint', 'wayne state u'),
 ('u mi, flint', 'fl atlantic u'),
 ('u mi, flint', 'u hong kong'),
 ('u mi, flint', 'social systems research institute'),
 ('u mi, flint', 'western new england college'),
 ('centre for development studies, medical college trivandrum',
  'centre for development studies, ulloor, india'),
 ('u antwerp-ufsia', 'u antwerp and erasmus u'),
 ('college o

In [20]:
len(affnwdict.edges())

64397

In [21]:
nx.info(affnwdict)

'Name: \nType: DiGraph\nNumber of nodes: 35091\nNumber of edges: 64397\nAverage in degree:   1.8351\nAverage out degree:   1.8351'

In [22]:
nx.write_gml(affnwdict,"trajectory_net.gml")